In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append("")

In [7]:
class M4Dataset(Dataset):
    """ Dataset for M4 AR(P) models """

    def __init__(self, arr, debug=False):
        if debug:
            np.random.shuffle(arr)
            self.X = torch.from_numpy(arr[:100, :])
        else:
            self.X = torch.from_numpy(arr)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx, :-1], self.X[idx, -1]

In [8]:
class MLP(nn.Module):
    def __init__(self, mem=2, n_hidden=8):
        super(MLP, self).__init__()
        self.h1 = nn.Linear(mem, n_hidden)
        self.h2 = nn.Linear(n_hidden, n_hidden)
        self.h3 = nn.Linear(n_hidden, n_hidden)
        self.h4 = nn.Linear(n_hidden, n_hidden)
        self.h5 = nn.Linear(n_hidden, 1)

    def forward(self, x):
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        x = F.relu(self.h3(x))
        x = F.relu(self.h4(x))
        x = self.h5(x)
        return x

In [9]:
class MLP2(nn.Module):
    """
    N layer multi layer perceptron with H hidden units in each layer.
    """
    def __init__(self, in_features:int, out_features:int=1, num_layers:int=5, n_hidden:int=32, bias:bool=True):
        super(MLP2, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.N = num_layers
        self.H = n_hidden
        self.memory = in_features
        self.bias=bias

        self.layers = nn.ModuleList()
        for i in range(num_layers-1):
            self.layers.append(nn.Linear(in_features=in_features if i==0 else n_hidden, out_features=n_hidden, bias=bias))
        self.out = nn.Linear(in_features=n_hidden, out_features=out_features, bias=bias)

        #self.init_weights()

    def forward(self, x):
        for i in range(self.N-1):
            x = F.relu(self.layers[i](x))
        out = self.out(x)
        return out
    
    def init_weights(self):
        for i in range(self.N - 1):
            nn.init.normal_(self.layers[i].weight, std=1e-3)
            if self.bias:
                nn.init.normal_(self.layers[i].bias, std=1e-6)
        nn.init.normal_(self.out.weight, std=1e-3)
        if self.bias:
            nn.init.normal_(self.out.bias, std=1e-6)

In [10]:
#np.random.seed(1729)
#torch.random.manual_seed(1729)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

debug = False

X_train = np.load("../M4_GLOBAL_train.npy")
X_test = np.load("../M4_GLOBAL_test.npy")

train = X_train[: int(X_train.shape[0] * 0.8), :]
val = X_train[int(X_train.shape[0] * 0.8) :, :]

#model = MLP(mem=12, n_hidden=32).double()
model = MLP2(in_features=12, out_features=1, num_layers=5, n_hidden=32).double()
model.to(device)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"{pytorch_total_params} trainable parameters in the network.")

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

ds = M4Dataset(arr=train, debug=False)
ds_val = M4Dataset(arr=val, debug=False)
trainloader = DataLoader(dataset=ds, batch_size=1024, shuffle=True, num_workers=0)
valloader = DataLoader(dataset=ds_val, batch_size=1024, shuffle=True, num_workers=0)

num_epochs = 1000
tenacity = 25
early_stop_count = 0
low_loss = np.inf
np.random.seed(1729)
torch.manual_seed(1729)
for ep in range(1, num_epochs + 1):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.flatten(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        model.eval()
    with torch.no_grad():
        val_loss = 0
        for i, data in enumerate(valloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs.flatten(), labels)
            val_loss += loss.item()
        if val_loss < low_loss:
            early_stop_count = 0
            low_loss = val_loss
        else:
            early_stop_count += 1
        if early_stop_count > tenacity:
            print(f"Early stop after epoch {ep}.")
            break
        # print(f"Epoch {epoch:3>} loss: {running_loss}")
    if ep % 10 == 0:
        print(f"Epoch {ep:3>} train loss: {running_loss}")
        print(f"Epoch {ep:3>} val loss  : {val_loss}")
        print(f"Early stop count = {early_stop_count}")
print("Finished training!")
with torch.no_grad():
    Y_hat = []
    for i in range(6):  # X_test.shape[1]
        if i == 0:
            X = X_train[:, i + 1 :]
        else:
            X = np.concatenate(
                (X_train[:, (i + 1) :], X_test[:, :i]), axis=1
            )

        sample = torch.from_numpy(X).to(device)
        out = model(sample).cpu().detach().numpy().flatten()
        Y_hat.append(out)

forecast = np.stack(Y_hat, axis=1)
# calculate mase (mae since we have already scaled)
error = np.mean(np.abs(forecast - X_test))
print(f"MASE : {error}")
error_axis = np.mean(np.mean(np.abs(forecast - X_test), axis=1))
print(f"MASE axis : {error_axis}")

forecast = np.stack(Y_hat, axis=1)
# calculate mase (mae since we have already scaled)
error = np.mean(np.abs(forecast[-23000:] - X_test[-23000:]))
print(f"MASE yearly: {error}")
error_axis = np.mean(np.mean(np.abs(forecast[-23000:] - X_test[-23000:]), axis=1))
print(f"MASE axis yearly : {error_axis}")

with torch.no_grad():
    for i in range(6):  # X_test.shape[1]
        sample = torch.from_numpy(X_train[:, -12:])
        out = model(sample).cpu().detach().numpy()
        X_train = np.hstack((X_train, out))

forecast = X_train[:, -6:]
error_axis = np.mean(np.mean(np.abs(forecast[-23000:] - X_test[-23000:]), axis=1))
print(f"MASE recursive yearly : {error_axis}")

cpu
3617 trainable parameters in the network.
Epoch 10 train loss: 61.938711143443065
Epoch 10 val loss  : 36.07511596692862
Early stop count = 0
Epoch 20 train loss: 58.1963314347814
Epoch 20 val loss  : 32.775944275808136
Early stop count = 0
Epoch 30 train loss: 56.31120307867703
Epoch 30 val loss  : 31.903761648488384
Early stop count = 6
Epoch 40 train loss: 55.7859607509825
Epoch 40 val loss  : 32.15770046339768
Early stop count = 4
Epoch 50 train loss: 57.19517395771521
Epoch 50 val loss  : 31.068936054665862
Early stop count = 5
Epoch 60 train loss: 53.75267256008897
Epoch 60 val loss  : 30.94843137158065
Early stop count = 2
Epoch 70 train loss: 51.28433543821948
Epoch 70 val loss  : 29.90660285066655
Early stop count = 2
Epoch 80 train loss: 51.68307633564861
Epoch 80 val loss  : 30.701617974942586
Early stop count = 5
Epoch 90 train loss: 50.8129419784539
Epoch 90 val loss  : 29.754989374230032
Early stop count = 0
Epoch 100 train loss: 51.3415619067958
Epoch 100 val loss  :

# Current train script

In [11]:
class MonteroMansoHyndmanSimpleDS(Dataset):
    """
    Dataset on the form of Monetero-Manso and Hyndman. Last 12:1 observations of each ts
    from M4 as samples. Last obs as label. Scaled by MASE.
    """
    def __init__(
        self, 
        arr:np.array,
        memory:int, 
        ) -> None:
        super(MonteroMansoHyndmanSimpleDS, self).__init__()
        self.X = torch.from_numpy(arr)
        
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx, :-1], self.X[idx, -1], 0

In [24]:
import sys
sys.path.append("")
sys.path.append("../../")

In [12]:
X_train = np.load("../M4_GLOBAL_train.npy")
X_test = np.load("../M4_GLOBAL_test.npy")

train = X_train[: int(X_train.shape[0] * 0.8), :]
val = X_train[int(X_train.shape[0] * 0.8) :, :]


#model = MLP(mem=12, n_hidden=32).double()
model = MLP2(in_features=12, out_features=1, num_layers=5, n_hidden=32).double()
print(f"Number of learnable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

#ds = MonteroMansoHyndmanSimpleDS(arr=train,memory=12,)
#ds_val = MonteroMansoHyndmanSimpleDS(arr=val, memory=12,)
ds = MonteroMansoHyndmanSimpleDS(arr=train, memory=12)
ds_val = MonteroMansoHyndmanSimpleDS(arr=val, memory=12)

trainloader = DataLoader(dataset=ds, batch_size=1024, shuffle=True, num_workers=0)
valloader = DataLoader(dataset=ds_val, batch_size=1024, shuffle=True, num_workers=0)

num_epochs = 1000
tenacity = 25
early_stop_count = 0
low_loss = np.inf
np.random.seed(1729)
torch.manual_seed(1729)
for ep in range(1, num_epochs + 1):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.flatten(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    with torch.no_grad():
        val_loss = 0
        for i, data in enumerate(valloader, 0):
            inputs, labels = data[0], data[1]
            outputs = model(inputs)
            loss = criterion(outputs.flatten(), labels)
            val_loss += loss.item()
        if val_loss < low_loss:
            early_stop_count = 0
            low_loss = val_loss
        else:
            early_stop_count += 1
        if early_stop_count > tenacity:
            print(f"Early stop after epoch {ep}.")
            break
        # print(f"Epoch {epoch:3>} loss: {running_loss}")
    if ep % 10 == 0:
        print(f"Epoch {ep:3>} train loss: {running_loss}")
        print(f"Epoch {ep:3>} val loss  : {val_loss}")
        print(f"Early stop count = {early_stop_count}")
print("Finished training!")

with torch.no_grad():
    Y_hat = []
    for i in range(6):  # X_test.shape[1]
        if i == 0:
            X = X_train[:, i + 1 :]
        else:
            X = np.concatenate((X_train[:, (i + 1) :], X_test[:, :i]), axis=1)
        sample = torch.from_numpy(X)
        out = model(sample).cpu().detach().numpy().flatten()
        Y_hat.append(out)

forecast = np.stack(Y_hat, axis=1)
# calculate mase (mae since we have already scaled)
error = np.mean(np.abs(forecast - X_test))
error_axis = np.mean(np.mean(np.abs(forecast - X_test), axis=1))
error_yearly = np.mean(np.abs(forecast[-23000:] - X_test[-23000:]))
error_yearly_axis = np.mean(np.mean(np.abs(forecast[-23000:] - X_test[-23000:]), axis=1))

print(f"MASE : {error}")
print(f"MASE axis : {error_axis}")
print(f"MASE yearly : {error_yearly}")
print(f"MASE axis yearly : {error_yearly_axis}")

# Testing yearly data recursive
df_yearly_train = pd.read_csv("../GPTime/data/raw/M4/M4train/Yearly-train.csv", index_col=0)
df_yearly_test = pd.read_csv("../GPTime/data/raw/M4/M4test/Yearly-test.csv", index_col=0)

scale = (df_yearly_train.diff(periods=1, axis=1).abs().mean(axis=1).reset_index(drop=True))

X_train_yearly = df_yearly_train.div(scale.values, axis=0).to_numpy()
X_test_yearly = df_yearly_test.div(scale.values, axis=0).to_numpy()

# X_train_yearly = df_yearly_train.to_numpy()
# X_test_yearly = df_yearly_test.to_numpy()
ts_train = []
ts_test = []
for i in range(X_train_yearly.shape[0]):
    s_train = X_train_yearly[i][~np.isnan(X_train_yearly[i])]
    s_test = X_test_yearly[i][~np.isnan(X_test_yearly[i])]
    ts_train.append(s_train[-12:])  # shortest in the train set
    ts_test.append(s_test[:6])  # shortest in the test set

df_train_out = pd.DataFrame(ts_train)
df_test_out = pd.DataFrame(ts_test)

X_train = np.array(ts_train)
X_test = np.array(ts_test)

print("recursive forecasting Yearly")

with torch.no_grad():
    for i in range(6):  # X_test.shape[1]
        sample = torch.from_numpy(X_train[:, -12:])
        out = model(sample).cpu().detach().numpy()
        X_train = np.hstack((X_train, out))

forecast = X_train[:, -6:]

error = np.mean(np.abs(forecast - X_test))
error_axis = np.mean(np.mean(np.abs(forecast - X_test), axis=1))
print(f"MASE Yearly recursive: {error}")
print(f"MASE Yearly axis recursive: {error_axis}")

X_train = np.array(ts_train)
X_test = np.array(ts_test)
print(X_train.shape)
print(X_test.shape)
print("forecasting Yearly")
with torch.no_grad():
    Y_hat = []
    for i in range(6):  # X_test.shape[1]
        if i == 0:
            X = X_train[:, i:]
        else:
            X = np.concatenate((X_train[:, i:], X_test[:, :i]), axis=1)
        sample = torch.from_numpy(X)
        out = model(sample).cpu().detach().numpy().flatten()
        Y_hat.append(out)

forecast = np.stack(Y_hat, axis=1)

error = np.mean(np.abs(forecast - X_test))
print(f"MASE Yearly one-step: {error}")
error_axis = np.mean(np.mean(np.abs(forecast - X_test), axis=1))
print(f"MASE Yearly_axis one-step: {error_axis}")

Number of learnable parameters: 3617
Epoch 10 train loss: 63.489465232711986
Epoch 10 val loss  : 35.28297568333003
Early stop count = 1
Epoch 20 train loss: 56.99212048817758
Epoch 20 val loss  : 32.19493199900279
Early stop count = 3
Epoch 30 train loss: 56.02792458158725
Epoch 30 val loss  : 31.078743880439536
Early stop count = 0
Epoch 40 train loss: 56.30400643171353
Epoch 40 val loss  : 31.438770832556678
Early stop count = 10
Epoch 50 train loss: 55.57690514028805
Epoch 50 val loss  : 31.168845647299236
Early stop count = 8
Epoch 60 train loss: 54.18120820517044
Epoch 60 val loss  : 29.4082656900572
Early stop count = 0
Epoch 70 train loss: 51.396107737226885
Epoch 70 val loss  : 30.765574373104354
Early stop count = 1
Epoch 80 train loss: 52.644673858308714
Epoch 80 val loss  : 30.04044787959961
Early stop count = 11
Epoch 90 train loss: 50.729309496860914
Epoch 90 val loss  : 29.40432281989407
Early stop count = 4
Epoch 100 train loss: 52.357727263124396
Epoch 100 val loss  : 